# Quantum Circuit Picker, Related Works

**Quantum CNN**
  * Pennylane Tutorial based on PyTorch
  * Build a quantum circuit and treat it as a quantum kernel
  * The quantum kernel consists of Pennylane's RY gates and RandomLayers
  * Ref: Quanvolutional Neural Networks — PennyLane; [1904.04767] Quanvolutional Neural Networks: Powering Image Recognition with Quantum Circuits (arxiv.org)


**Quantum GAN**
  * Pennylane Tutorial based on Google Cirq and tensorflow
  * Build a generator and a discriminator that consist of Pennylane's Hadamard, RX, RY, RZ, CNOT gates
  * Ref: Quantum Generative Adversarial Networks with Cirq + TensorFlow — PennyLane


**Quantum Neural Nets**

  * Pennylane Tutorial based on Google Cirq and tensorflow
  * Build a generator and a discriminator that consist of Pennylane's Hadamard, RX, RY, RZ, CNOT gates
  * Ref: Quantum Generative Adversarial Networks with Cirq + TensorFlow — PennyLane
  
**Single Qubit Classifier**

  * Pennylane Tutorial based on Pennylane
  * Build a circuit that consists of ROT gates
  * Ref: Data-reuploading classifier — PennyLane
  
**Transfer Learning using ANN**

  * Pennylane Tutorial based on Pennylane and PyTorch
  * Build quantum layers of pennylane's Hadamard, RY, CNOT gates
  * Ref: Quantum transfer learning — PennyLane
  
**Variational Quantum Linear Solver**

  * Pennylane Tutorial based on Pennylane 
  * Build quantum circuits that consist of pennylane's Hadamard, CZ, CNOT gates
  * Ref: Variational Quantum Linear Solver — PennyLane
  
**Coherent Variational Quantum Linear Solver**

  * Pennylane Tutorial based on Pennylane 
  * Build quantum circuits that consist of pennylane's Hadamard, CZ, CNOT, CRY, RY gates
  * Ref: https://pennylane.ai/qml/demos/tutorial_coherent_vqls.html
  
**Multiclass Classification**

  * Pennylane Tutorial based on Pennylane 
  * Build quantum circuits that consist of pennylane's ROT, CNOTgates
  * Ref: Multiclass margin classifier — PennyLane

**QGRNN**

  * Pennylane Tutorial based on Pennylane 
  * Build quantum circuits that consist of pennylane's ROT, CNOTgates
  * Ref: Multiclass margin classifier — PennyLane


In [1]:
interface = "torch" # pytorch
simulator = "default.qubit" # cirq.simulator
nntype = "ann" # gan, cnn, gnn, anntf (ann for transfer learning), lstm

In [ ]:

switch (nntype) {
    case 'ann':
        qlayer = Ann1QbitLayer()
        break;
    case 'cnn':
        qlayer = CnnLayer()
        break;
    case 'anntf':
        qlayer = AnnTfLayer()
        break;
    case 'gan':
        print('GAN quantum circuit picker under development...')
        break;
    case 'gnn':
        print('GNN quantum circuit picker under development...')
        break;
    default:
        print('invalid neural net type')
        break;
}

In [ ]:
class Ann1QbitLayer(dev, params, features, density):
    
    @qml.qnode(dev)
    def qcircuit(params, features, density):
        """A variational quantum circuit representing the Universal classifier.

        Args:
            params (array[float]): array of parameters
            features (array[float]): single input vector
            density (array[float]): single output state density matrix

        Returns:
            float: fidelity between output state and input
        """
        for p in params:
            qml.Rot(*features, wires=0)
            qml.Rot(*p, wires=0)
        return qml.expval(qml.Hermitian(density, wires=[0]))

In [ ]:
# ref: https://github.com/PlanQK/TrainableQuantumConvolution/blob/main/Demo%20Trainable%20Quantum%20Convolution.ipynb
class CnnLayer():
        @qml.qnode(device=self.dev, interface="torch")
        def circuit(inputs, weights): # a quantum kernel
            n_inputs=4
            # Encoding of 4 classical input values
            for j in range(n_inputs):
                qml.RY(inputs[j], wires=j)
            # Random quantum circuit
            RandomLayers(weights, wires=list(range(self.wires)), seed=seed)
            
            # Measurement producing 4 classical output values
            return [qml.expval(qml.PauliZ(j)) for j in range(self.out_channels)]

In [ ]:
# ref example: https://pennylane.readthedocs.io/en/latest/_modules/pennylane/qnn/keras.html#KerasLayer
# ref: https://pennylane.ai/qml/demos/tutorial_quantum_transfer_learning.html
class AnnTfLayer(dev, wires, q_input_features, q_weights_flat):
    
    nqubits = len(wires)
    
    def H_layer(nqubits):
    """Layer of single-qubit Hadamard gates.
    """
    for idx in range(nqubits):
        qml.Hadamard(wires=idx)


    def RY_layer(w):
        """Layer of parametrized qubit rotations around the y axis.
        """
        for idx, element in enumerate(w):
            qml.RY(element, wires=idx)


    def entangling_layer(nqubits):
        """Layer of CNOTs followed by another shifted layer of CNOT.
        """
        # In other words it should apply something like :
        # CNOT  CNOT  CNOT  CNOT...  CNOT
        #   CNOT  CNOT  CNOT...  CNOT
        for i in range(0, nqubits - 1, 2):  # Loop over even indices: i=0,2,...N-2
            qml.CNOT(wires=[i, i + 1])
        for i in range(1, nqubits - 1, 2):  # Loop over odd indices:  i=1,3,...N-3
            qml.CNOT(wires=[i, i + 1])
    
    @qml.qnode(dev, interface="torch")
    def quantum_net(q_input_features, q_weights_flat):
        """
        The variational quantum circuit.
        """

        # Reshape weights
        q_weights = q_weights_flat.reshape(q_depth, n_qubits)

        # Start from state |+> , unbiased w.r.t. |0> and |1>
        H_layer(n_qubits)

        # Embed features in the quantum node
        RY_layer(q_input_features)

        # Sequence of trainable variational layers
        for k in range(q_depth):
            entangling_layer(n_qubits)
            RY_layer(q_weights[k])

        # Expectation values in the Z basis
        exp_vals = [qml.expval(qml.PauliZ(position)) for position in range(n_qubits)]
        return tuple(exp_vals)